# 文章の要約（LexRank）
参考⇒https://ohke.hateblo.jp/entry/2018/11/17/230000 。分かり易い⇒
http://maruo51.com/2019/07/07/python-sumy/

In [ ]:
from janome.analyzer import Analyzer
from janome.charfilter import UnicodeNormalizeCharFilter, RegexReplaceCharFilter
from janome.tokenizer import Tokenizer as JanomeTokenizer  # sumyのTokenizerと名前が被るため
from janome.tokenfilter import POSKeepFilter, ExtractAttributeFilter

f = open('年金_約款.txt',encoding='utf-8')
text = f.read()  # ファイル終端まで全て読んだデータを返す
f.close()

# 1行1文となっているため、改行コードで分離
sentences = [t for t in text.split('\n')]
analyzer = Analyzer(
    [UnicodeNormalizeCharFilter(), RegexReplaceCharFilter(r'[(\)「」、。]', ' ')],  # ()「」、。は全てスペースに置き換える
    JanomeTokenizer(),
    [POSKeepFilter(['名詞', '形容詞', '副詞', '動詞']), ExtractAttributeFilter('base_form')]  # 名詞・形容詞・副詞・動詞の原型のみ
)

corpus = [' '.join(analyzer.analyze(s)) + '。' for s in sentences]
for i in range(4):
    print(corpus[i])

In [74]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

# 連結したcorpusを再度tinysegmenterでトークナイズさせる
parser = PlaintextParser.from_string(''.join(corpus), Tokenizer('japanese'))

# LexRankで要約を2文抽出
summarizer = LexRankSummarizer()
summarizer.stop_words = [' ']  # スペースも1単語として認識されるため、ストップワードにすることで除外する

summary = summarizer(document=parser.document, sentences_count=1)

# 元の文を表示
for sentence in summary:
    print(sentences[corpus.index(sentence.__str__())])

●この保険は、被保険者が所定の年齢になられたときから年金をお支払いします。なお、年金支払開始日前に死亡され


In [76]:
print(summary)

(<Sentence: 保険 保険 者 所定 年齢 なる れる とき 年金 支払い する 年金 支払 開始 日 前 死亡 する れる。>,)
